In [2]:
import cv2
import numpy as np
import math
import os
from PIL import ImageGrab
import os.path as osp
from ultralytics import YOLO, RTDETR
import pydirectinput
import time

# load model
model = YOLO("./train9 - yolo5s.pt")

# 화면 크기 설정
screen_size = (1280, 720)

# 시야각
fov = 103
angrat=math.tan(math.radians(fov / 2))

# 중앙 300x300 영역 설정
center_size=300
center_half=center_size/2
center_x = (screen_size[0] - center_size) // 2
center_y = (screen_size[1] - center_size) // 2
bbox = (center_x, center_y, center_x + center_size, center_y + center_size)

# 저장 경로 디폴트셋팅
train_path = "images/"
label_path = "labels/"

# 이미지 저장할 폴더 생성
if not osp.isdir(train_path):
    os.makedirs(train_path)

# 텍스트 파일 저장할 폴더 생성
if not osp.isdir(label_path):
    os.makedirs(label_path)


# 파일 열기 함수
def open_file(filename, mode):
    try:
        return open(filename, mode)
    except FileNotFoundError:
        return open(filename, "w")

# 폴더 내 파일 개수 확인
def file_count(path):
    global count
    folder_path = path
    file_list = os.listdir(folder_path)
    count = len(file_list)
    return count

# 박스의 w/2,h/2 는 size의 크기와 같도록 함
size=5

count = file_count(train_path)

# 클릭 좌표 관련 변수
pointlist=[]
# 캐릭터별 번호
hero_num=1
# 라벨링 모드
menu=""
# 상태 관련 bool
while_rec=False
charic_select=False

# 마우스 이벤트
def mouse_callback(event, x, y, flags, param):
    global pointlist,  charic_select
    global menu, size, hero_num

    # 좌클릭시 pointlist에 현재 캐릭터, 좌표 추가
    if event == cv2.EVENT_LBUTTONDOWN and while_stop==True:
        point = [x,y,size,hero_num]
        pointlist.append(point)
        if charic_select==True:
            charic_select=False
            menu=""
    
    elif event == cv2.EVENT_MOUSEWHEEL:
        # 캐릭터 종류 변경
        if charic_select==True:
            if flags > 0:
                hero_num += 1
            else:
                hero_num -= 1
        else:
            # 마우스 휠에 따라 사각형 크기 조절
            if flags > 0:
                size += 1
            elif size > 1:
                size -= 1
        if hero_num>39:
                hero_num=1
        elif hero_num<1:
            hero_num=39

heroes={1: 'mauga', 2: 'ana', 3: 'ashe', 4: 'baptiste', 5: 'bastion', 6: 'brigitte', 7: 'cassidy', 8: 'd.va', 9: 'doomfist', 10: 'echo',
        11: 'genji',12: 'hanzo', 13: 'illari', 14: 'junkerqeen', 15: 'junkrat', 16: 'kiriko', 17: 'lifeweaver', 18: 'lucio', 19: 'mei', 20: 'mercy',
        21: 'moira', 22: 'orisa', 23: 'pharah', 24: 'ramattra', 25: 'reaper', 26: 'reinhardt', 27: 'roadhog', 28: 'sigma', 29: 'sojourn', 30: 'soldier:76',
        31: 'sombra', 32: 'symmetra', 33: 'torbjorn', 34: 'tracer', 35: 'widowmaker', 36: 'winston', 37: 'wreckingball', 38: 'zarya', 39: 'zenyatta'}

c:\Users\MHJ\Desktop\github\MachineLearningProject\.test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


녹화

In [3]:
# 녹화할 파일명과 코덱 설정
output_file = 'center_screen_recording_alone.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# 비디오 녹화 객체 생성
out = cv2.VideoWriter(output_file, fourcc, 20.0, (center_size, center_size))
while True:
    # 현재 화면 캡쳐
    img = ImageGrab.grab(bbox=bbox)
    # 화면에 표시 및 파일에 저장
    img=np.array(img)
    cv2.imshow('Center Screen Recording', img)
    out.write(img)
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break
# 사용한 자원 해제
out.release()
cv2.destroyAllWindows()

YOLO용 라벨링 함수(클래스, x, y, w, h)

In [2]:
# 마우스 클릭 좌표 기준 사각형 그리기
def stopped():
    global train_path, count
    global pointlist, while_stop, charic_select, menu
    while_stop=True

    while True:
        key=cv2.waitKey(10)
        tempimg=img.copy()

        # 상태바 & 박스 그리기
        cv2.putText(tempimg, f"{menu}stop count:{count} \n tag : {heroes[hero_num]}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
        if len(pointlist)>0:
            for point in pointlist:
                cv2.rectangle(tempimg, (point[0]-point[2], point[1]-point[2]), (point[0]+point[2], point[1]+point[2]), (255-point[3]*6, point[3]*6, -abs(6*(point[3]-20))+120))
        cv2.imshow("win",tempimg)

        if key == ord(" "):
            if  len(pointlist)>0:
                count = file_count(train_path)
                text = open_file(label_path + str(count) + ".txt", "w")
                cv2.imwrite(train_path + str(count) + '.jpg', img)
                for point in pointlist:
                    text.write(f"{point[3]} {float(point[0])/center_size} {float(point[1])/center_size} {float(point[2])*2/center_size} {float(point[2])*2/center_size}")
                    text.write("\n")
                text.close()
            break

        # 마지막 박스 지우기
        elif key == ord("x"):
            if len(pointlist)>0:
                pointlist.pop()
        # 캐릭터 변경 모드
        elif key == ord("z"):
            charic_select=True
            menu="charic change-"

    #변수 초기화
    while_stop=False
    pointlist=[]


simple모드 (x,y좌표만)

In [4]:
# 마우스 클릭 좌표 기준 사각형 그리기
def stopped():
    global train_path, count
    global pointlist, while_stop, charic_select, menu
    while_stop=True

    while True:
        key=cv2.waitKey(10)
        tempimg=img.copy()

        # 상태바 & 박스 그리기
        cv2.putText(tempimg, f"{menu}stop count:{count} \n tag : {heroes[hero_num]}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
        if len(pointlist)>0:
            for point in pointlist:
                cv2.rectangle(tempimg, (point[0]-point[2], point[1]-point[2]), (point[0]+point[2], point[1]+point[2]), (255-point[3]*6, point[3]*6, -abs(6*(point[3]-20))+120))
        cv2.imshow("win",tempimg)

        if key == ord(" "):
            if  len(pointlist)>0:
                count = file_count(train_path)
                text = open_file(label_path + str(count) + ".txt", "w")
                cv2.imwrite(train_path + str(count) + '.jpg', img)
                for point in pointlist:
                    text.write(f"{float(point[0])/center_size} {float(point[1])/center_size}")
                    text.write("\n")
                text.close()
            break

        # 마지막 박스 지우기
        elif key == ord("x"):
            if len(pointlist)>0:
                pointlist.pop()
        # 캐릭터 변경 모드
        elif key == ord("z"):
            charic_select=True
            menu="charic change-"

    #변수 초기화
    while_stop=False
    pointlist=[]

라벨링

D: 60프레임 이후로 스킵
A: 60프레임 이전으로 스킵
Space: 화면 정지
좌클릭: 중앙점 생성, 정사각형 생성
횔업&다운: 정사각형 크기 늘리고 줄이고
Z: 캐릭터 선택
X: 최근에 선택한 박스 취소

In [ ]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
cv2.setMouseCallback("win", mouse_callback)
ret,img=readvideo.read()

while ret:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    key = cv2.waitKey(33)
    
    # 프레임 조정
    if key == ord("d"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) + 60)
    elif key == ord("a"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) - 60)

    # 사각형 생성
    elif key == ord(" "):
        stopped()

    # 영상 종료
    elif key == ord("q"):
        break
    
    ret,img=readvideo.read()

    cv2.imshow("win", img)

readvideo.release()
cv2.destroyAllWindows()

마우스 이동(YOLO)

In [5]:
# 비디오 캡쳐
while True:
    img = ImageGrab.grab(bbox=bbox)
    results=model(img)
    for r in results:
        if len(r.boxes.xywh)>0:
                result = r.boxes.xywh[0].cpu().numpy()
                result = result.tolist()
                x, y, w, h= result
                #디폴트
                #pydirectinput.moveRel(round(x-150), round(y-150), relative=True)
                #제곱
                #pydirectinput.moveRel(round((x-150)*abs(x-150)/80), round((y-150)*abs(x-150)/80), relative=True)
                #아크탄젠트
                #pydirectinput.moveRel(round(math.atan(x-150)*abs(x-150)), round(math.atan(y-150)*abs(y-150)), relative=True)
                #시야각형식+속도 고려
                #pydirectinput.moveRel(round((x-center_half)*max(abs(math.atan(x-center_half)),1) / angrat), round((y-center_half)*max(abs(math.atan(y-center_half)),1) / angrat), relative=True)
                L=(640/math.tan(math.radians(103/2)))
                xx=math.atan((x-center_half)/L)
                yy=math.atan((y-center_half)/L)
                print(xx,yy)
                pydirectinput.moveRel(round(xx*L),round((yy*L)), relative=True)


0: 160x160 (no detections), 86.1ms
Speed: 4.0ms preprocess, 86.1ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 111.2ms
Speed: 0.0ms preprocess, 111.2ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 97.1ms
Speed: 1.0ms preprocess, 97.1ms inference, 1.3ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 90.6ms
Speed: 1.0ms preprocess, 90.6ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 80.4ms
Speed: 2.0ms preprocess, 80.4ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 96.8ms
Speed: 0.0ms preprocess, 96.8ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 93.8ms
Speed: 3.0ms preprocess, 93.8ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 83.7ms
Speed: 1.0ms preprocess, 83.7ms

KeyboardInterrupt: 

CNN 데이터셋 라벨링 확인용

In [ ]:
# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./images"
lth = "./labels"

# 이미지 목록과 레이블 목록을 생성
img_list = os.listdir(pth)
lab_list = os.listdir(lth)

# 이미지와 레이블을 읽음
for i in range(len(img_list)):
    if i ==653:
        continue
    im = cv2.imread(pth + '/' + str(i) + ".jpg")
    with open(lth + '/' + str(i) + ".txt","r") as lbr:
        ll=lbr.read()
        ll=list(ll.split())
    ll=[round(center_size*float(x)) for x in ll]    
    cv2.circle(im, (ll[0],ll[1]), 5, (0,255,0), 2)
    # 텍스트 추가
    cv2.putText(im, str(i), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 이미지 출력
    cv2.imshow("chek", im)
    cv2.waitKey(0)

# 모든 창 닫음
cv2.destroyAllWindows()